<a href="https://colab.research.google.com/github/koad7/NLP_PYTORCH/blob/main/Word2VecCustomModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Basic word2vec functionality using the google word2vec model Code

In [16]:
import gensim.downloader as api
import json
import pandas as pd
import string
import time
import requests
import gzip
import spacy

In [19]:
import string

In [3]:
w2v = api.load('word2vec-google-news-300')

In [4]:
for i, word in enumerate(w2v.key_to_index):
    if i == 10:
        break
    print(word)

</s>
in
for
that
is
on
##
The
with
said


The dataset is from Amazon Review Data (2018) https://nijianmo.github.io/amazon/index.html.

In [5]:
### load the meta data
!wget --no-check-certificate https://jmcauley.ucsd.edu/data/amazon_v2/categoryFilesSmall/Cell_Phones_and_Accessories_5.json.gz


--2023-07-05 14:18:21--  https://jmcauley.ucsd.edu/data/amazon_v2/categoryFilesSmall/Cell_Phones_and_Accessories_5.json.gz
Resolving jmcauley.ucsd.edu (jmcauley.ucsd.edu)... 137.110.160.73
Connecting to jmcauley.ucsd.edu (jmcauley.ucsd.edu)|137.110.160.73|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 169071325 (161M) [application/x-gzip]
Saving to: ‘Cell_Phones_and_Accessories_5.json.gz’

Cell_Phones_and_Acc 100%[===================>] 161.24M  22.6MB/s    in 8.0s    

2023-07-05 14:18:29 (20.3 MB/s) - ‘Cell_Phones_and_Accessories_5.json.gz’ saved [169071325/169071325]



In [9]:
data = []
with gzip.open('Cell_Phones_and_Accessories_5.json.gz') as f:
    for l in f:
        data.append(json.loads(l.strip()))

In [10]:
print(data[0])
df = pd.DataFrame(data)
print(len(data))

{'overall': 5.0, 'verified': True, 'reviewTime': '08 4, 2014', 'reviewerID': 'A24E3SXTC62LJI', 'asin': '7508492919', 'style': {'Color:': ' Bling'}, 'reviewerName': 'Claudia Valdivia', 'reviewText': 'Looks even better in person. Be careful to not drop your phone so often because the rhinestones will fall off (duh). More of a decorative case than it is protective, but I will say that it fits perfectly and securely on my phone. Overall, very pleased with this purchase.', 'summary': "Can't stop won't stop looking at it", 'unixReviewTime': 1407110400}
1128437


In [11]:
df = df.drop(columns=['reviewerName', 'vote','image','style'])
df1 = df.rename(columns = {'overall': 'rating', 'asin': 'productID'}, inplace = False)

In [13]:
df1.dropna(axis = 0, how ='any',inplace=True)
df1.drop_duplicates(subset=['rating','reviewText'],keep='first',inplace=True)

In [14]:
df1.head()

,rating,verified,reviewTime,reviewerID,productID,reviewText,summary,unixReviewTime
0,5.0,True,"08 4, 2014",A24E3SXTC62LJI,7508492919,Looks even better in person. Be careful to not...,Can't stop won't stop looking at it,1407110400
1,5.0,True,"02 12, 2014",A269FLZCB4GIPV,7508492919,When you don't want to spend a whole lot of ca...,1,1392163200
2,3.0,True,"02 8, 2014",AB6CHQWHZW4TV,7508492919,"so the case came on time, i love the design. I...",Its okay,1391817600
3,2.0,True,"02 4, 2014",A1M117A53LEI8,7508492919,DON'T CARE FOR IT. GAVE IT AS A GIFT AND THEY...,CASE,1391472000
4,4.0,True,"02 3, 2014",A272DUT8M88ZS8,7508492919,"I liked it because it was cute, but the studs ...",Cute!,1391385600


In [22]:
nlp = spacy.load("en_core_web_sm")
stop_words = nlp.Defaults.stop_words
punctuations = string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [24]:
def clean_text(text ):
    """
    This function cleans the input text by removing punctuation, digits, and words that have a length of 2 or less.

    Args:
    text (str): The input string to be cleaned.

    Returns:
    list: A list of cleaned, lower-cased words from the input string.
    """
    delete_dict = {sp_character: ' ' for sp_character in string.punctuation}
    delete_dict[' '] = ' '
    table = str.maketrans(delete_dict)
    text1 = text.translate(table)
    #print('cleaned:'+text1)
    textArr= text1.split()
    text2 = ' '.join([ word for word in textArr if ( not word.isdigit() and (word not in stop_words and word not in punctuations)) ])

    return text2.lower().split(' ')

In [25]:
df2 = df1.sample(n=200000)
df2['reviewText']= df2['reviewText'].apply(clean_text)